# Portfolio Optimization with Genetic Programming

# 1. Defining the terminals
In this first step we define the Primitive set (`pset`). The `pset` contains the functions, constants and input parameters our tree will be build of.

First define the input parameters for the generated trees.

In [ ]:
from deap import base, creator, tools, gp, algorithms
import operator, math, random, csv
from stock import Stock
import pandas as pd
import numpy as np
from tree_plot import *
from primitive_functions import *
from parser import *
from dateutil.relativedelta import relativedelta

pset = gp.PrimitiveSetTyped("main", [Stock], int)

Define primitive functions for the nodes. The first parameter defines the function, the second the types of the input parameters and the third one the type of the output.

In [ ]:
pset.addPrimitive(profit10, [Stock, int, int], int, name="profit_larger_10")
pset.addPrimitive(profit20, [Stock, int, int], int, name="profit_larger_20")
pset.addPrimitive(profit40, [Stock, int, int], int, name="profit_larger_40")
pset.addPrimitive(profit60, [Stock, int, int], int, name="profit_larger_60")
pset.addPrimitive(profit80, [Stock, int, int], int, name="profit_larger_80")
pset.addPrimitive(risk10, [Stock, int, int], int, name="risk_larger_10")
pset.addPrimitive(risk20, [Stock, int, int], int, name="risk_larger_20")
pset.addPrimitive(risk40, [Stock, int, int], int, name="risk_larger_40")
pset.addPrimitive(risk60, [Stock, int, int], int, name="risk_larger_60")
pset.addPrimitive(risk80, [Stock, int, int], int, name="risk_larger_80")
pset.addPrimitive(stock, [Stock], Stock, name="asset_fun")


pset.addTerminal(0, int)
pset.addTerminal(12, int)
pset.addTerminal(24, int)
pset.addTerminal(48, int)

Renaming the input parameter

In [ ]:
pset.renameArguments(ARG0="asset")

# 2. Defining Object types
In any evolutionary program, we need some basic object types. In this case we need two, a fitness type and the type for individuals. In this problem we are facing an maximization and minimization problem (maximize the value and minimize the tree size)(so the one value is positive and the other one is negative). The individual will be based upon a tree, to which we add the defined fitness.

In [ ]:
creator.create("Fitness", base.Fitness, weights=(1.0, -1.0, ))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.Fitness, pset=pset)

# 3. Define helper functions
Register functions that we during the whole algorithm (generate, evaluate, mutate, ...). Any structure with access to the toolbox will also have access to all of those registered parameters.

## 3.1 Generating individuals

In [ ]:
toolbox = base.Toolbox()
# Defines how a tree expression looks like
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
# How an individual should be generated (in this case as a tree)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
# How the population of individuals should look like
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## 3.2 Evaluation of trees
Define functions that help us to evaluate an individuum. This includes to calculate the fitness. But first we need to generate executable pythoncode out of our tree individuals.

To get working python code out of our generated tree we can use the `gp.compile` function.

In [ ]:
# Generates Python code out of trees
toolbox.register("compile", gp.compile, pset=pset)

all_symbols = load_all_symbols()
all_assets = generate_all_assets(all_symbols, sample=len(all_symbols))

def evaluate(tree):
    # using the previously defined compile function
    function = toolbox.compile(tree)
    # Get all asset symbols as a list of strings
    assets = all_assets

    # Set start and end date which is used to generate the trees
    end = datetime(2015, 1, 23)

    # Following part is used to evaluate the trees
    profit = 0.0
    risk = 0.0
    # Go through all assets and get a recommendation for how long one should hold the stock, based on the historical data
    for asset in random.sample(assets, 10):
        hold_duration = function(asset)
        if hold_duration is not None and hold_duration > 0:
            # Get the profit and risk for the calculated hold time for one specific asset
            future_profit = asset.get_avg_profit(start=end, end=end + relativedelta(months=hold_duration))
            future_risk = asset.get_avg_risk(start=end, end=end + relativedelta(months=hold_duration))
            if future_profit is not None and future_profit is not np.nan:
                profit += future_profit
            if future_risk is not None and future_risk is not np.nan:
                risk += future_risk

    profit = profit / len(all_assets)
    risk = risk / len(all_assets)
    return profit, risk,

# Now add the evaluation Function to our toolbox
toolbox.register("evaluate", evaluate)

## 3.3 Selection for next generation
We choose the selection method (a tournament of size 3).

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=3)

## 3.4 Mutation
Define random mutations for individuals. 

In [ ]:
# Defines how the expression of a tree mutation should look like
toolbox.register("expr_mut", gp.genFull, min_=0, max_=4)
# How the mutation should be applied
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
# Limit the height of individual to avoid bloat
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

# Crossover
toolbox.register("mate", gp.cxOnePointLeafBiased, termpb=0.2)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

# 4. Add some statistics
 In this case, we want to compute the mean, standard deviation, minimum, and maximum of both the individuals fitness and size.

In [ ]:
def stats():
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    return mstats

# 5. Plott trees

In [ ]:
def plot(trees):
    write_file(trees[0])
    for index, tree in zip(range(len(trees)), trees):
        # Generate Graph
        print("generate Graph for tree " + str(index))
        nodes, edges, labels = gp.graph(tree)
        plot_tree(nodes, edges, labels, 'tree' + str(index))

# 6. Launch

In [ ]:
pop = toolbox.population(n=10)
hof = tools.HallOfFame(1)

mstats = stats()

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.3, mutpb=0.2, ngen=25, halloffame=hof, stats=mstats, verbose=True)


plot(pop)

# Display the tree
from IPython.display import Image
Image(filename="tree0.png")